In [1]:
import numpy as np
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("..")

In [2]:
import torch
from calflops import calculate_flops
import sophius.utils as utils
import sophius.dataload as dload
from sophius.modelgen import ConvModelGenerator
from sophius.train import train_express_gpu
import torchvision.datasets as dset
import torchvision.transforms as T
from sophius.encode import Encoder

VAL_SIZE = 1024

cifar10 = dset.CIFAR10('../data/CIFAR10', train=True, download=True,
                           transform=T.ToTensor())
cifar_gpu = dload.cifar_to_gpu(cifar10)

encoder = Encoder()

C:\Users\alexander.korolyov\miniforge3\envs\sophius\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Files already downloaded and verified


In [3]:
model_gen = ConvModelGenerator((3, 32, 32), 10, conv_num=6, lin_num=3)
model_tmpl = model_gen.generate_model_tmpl()
print(model_tmpl)
fixed_model_gpu = model_tmpl.instantiate_model().type(torch.cuda.FloatTensor)
print(encoder.encode_model(model_tmpl))

Conv2d       (64, 8, 8)     (5, 5)   (4, 4)  
PReLU        (64, 8, 8)    
Conv2d       (192, 3, 3)    (4, 4)   (3, 3)  
MaxPool2d    (192, 1, 1)    (3, 3)   (3, 3)  
LeakyReLU    (192, 1, 1)    (0.01)  
Flatten      192           
Linear       1024          
Dropout      1024           (0.75)  
ReLU         1024          
Linear       256           
ReLU         256           
Dropout      256            (0.85)  
Linear       10            

[[0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 1 1 0]
 [0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 1 0 1 0]
 [0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0]
 [0 0 1 0 0 0 0 0 

In [4]:
model_tmpl = model_gen.generate_model_tmpl()
vec = encoder.encode_model(model_tmpl)
vec.dtype

dtype('uint8')

In [46]:
import numpy as np

model_tmpl = model_gen.generate_model_tmpl()
arr = encoder.encode_model(model_tmpl)

hex_str = encoder.encode_model_str(model_tmpl)

print(arr)
print(hex_str)

[[0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 1 0 0 1 0]
 [0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 1 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 1 0]
 [0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 1]
 [0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]
00802092100800000022d0000084008a100200004000000000810209010000008004000020000000


In [6]:
import sqlite3
import pandas as pd
import seaborn as sns
from tqdm import tqdm

loader_gpu = dload.get_loader_gpu(cifar_gpu, val_size=VAL_SIZE, batch_size=1024)

df = pd.DataFrame(columns=['hex_str', 'flops', 'macs', 'params', 'time', 'val_acc', 'train_acc'])

for i in tqdm(range(10)):
    model_tmpl = model_gen.generate_model_tmpl()
    model = model_tmpl.instantiate_model().type(torch.cuda.FloatTensor)
    flops, macs, params = calculate_flops(model=model, 
                                      input_shape=(1, 3, 32, 32),
                                      output_as_string=False,
                                      print_results=False,
                                      print_detailed=False,
                                      output_precision=4)
    # express train
    t, val_acc, train_acc = train_express_gpu(model = model,
                                              train = True,
                                              loader = loader_gpu,
                                              milestones = [],
                                              num_epoch = 1,
                                              verbose = False)
    
    hex_str = encoder.encode_model_str(model_tmpl)
    
    df.loc[i] = [hex_str, flops, macs, params, t, val_acc, train_acc]
    
    with sqlite3.connect('../data/models.db') as conn:
        df.loc[i:i].to_sql('test', conn, if_exists='append')
    
    # print(df.loc[i])

100%|██████████| 10/10 [00:24<00:00,  2.48s/it]


In [7]:
df

,hex_str,flops,macs,params,time,val_acc,train_acc
0,00880111004490001008000001000000,244384,120480,3498,1.021995,0.296875,0.291016
1,0084010a00293000200000004000000000820812400000...,768704,376624,125355,0.899617,0.314453,0.296875
2,008100610024b000004920002000000000810049080400...,16529904,8209664,254947,2.692581,0.237305,0.204102
3,0080411220000000002530004000000000804406080800...,13018368,6430720,1775882,3.647358,0.446289,0.453125
4,008084090022b000200000000080300920000000010000...,3264768,1619072,1311818,0.925311,0.267578,0.251953
5,00804052400000000022c8001008000000880092004250...,11701816,5779776,97523,3.601307,0.254883,0.273438
6,0084020920000000004510000084021100295000400000...,244448,118992,6450,0.298085,0.174805,0.166992
7,0088005108040000002330000010000001000000800200...,971816,482392,289068,0.393367,0.235352,0.227539
8,0088101240000000100200000081008600446000400000...,5316112,2649600,2186347,1.025380,0.369141,0.391602
9,0082020a00253000200000004000000000820409200000...,560032,271680,13834,0.426872,0.342773,0.354492


In [44]:
import numpy as np

from sophius.encode import str_to_vec, vec_to_str

In [45]:
for i in tqdm(range(10000)):
    model_tmpl = model_gen.generate_model_tmpl()
    orig_arr = encoder.encode_model(model_tmpl)
    
    hex_str = vec_to_str(orig_arr)
    arr = str_to_vec(hex_str)
    
    assert np.array_equal(arr, orig_arr)

100%|██████████| 10000/10000 [00:06<00:00, 1514.10it/s]
